In [1]:
from scipy.stats import skew
from IPython.display import display
import matplotlib.pyplot as plt
import sys
sys.path.append('/Users/yanhuan/anaconda/pkgs/seaborn-0.7.1-py27_0/lib/python2.7/site-packages')
#import seaborn as sns
import numpy as np
import pandas as pd


SP500=pd.read_csv('SP500_new.csv')
Nasdaq=pd.read_csv('Nasdaq_new.csv')
DJI=pd.read_csv('DJI_new.csv')
DAX=pd.read_csv('DAX_new.csv')
Paris=pd.read_csv('Paris_new.csv')
Tokyo=pd.read_csv('Tokyo_new.csv')
HongKong=pd.read_csv('HongKong_new.csv')
Aus=pd.read_csv('Aus_new.csv')

Data Preprocess

In [2]:
Target=SP500['Adj Close'].pct_change(1).shift(-1)
datasets=[SP500,Nasdaq,DJI,DAX,Paris,Tokyo,HongKong,Aus]
namesets=['SP500','Nasdaq','DJI','DAX','Paris','Tokyo','HongKong','Aus']
def addFeatures(datasets,DaysRet,DaysRetMovAvg,DaysLags):
    """
    compute dataset's DaysReturns,DaysReturnMovAvg and DaysLags
    DaysReturnMovAvg must >=2
    """
    Max=max(DaysRet,DaysRetMovAvg,DaysLags+1)
    for i in range(len(datasets)):
        dataset=datasets[i]
        nameset=namesets[i]
        for i in range(1,DaysRet+1):
            dataset[nameset+'_'+str(i)+'_'+'DaysRet'] = dataset['Adj Close'].pct_change(i)
        for i in range(2,DaysRetMovAvg+1):
            dataset[nameset+'_'+str(i)+'_'+'DaysRetMovAvg'] = pd.rolling_mean(dataset[nameset+'_'+str(1)+'_'+'DaysRet'], i)
        for i in range(1,DaysLags+1):
            dataset[nameset+'_'+str(i)+'_'+'DaysLags'] = dataset[nameset+'_'+str(1)+'_'+'DaysRet'].shift(i)
        dataset=dataset[Max:]
    return Max

In [3]:
Max=addFeatures(datasets,9,9,9)

C:\Program Files\Anaconda3\lib\site-packages\ipykernel\__main__.py:16: FutureWarning: pd.rolling_mean is deprecated for Series and will be removed in a future version, replace with 
	Series.rolling(window=2,center=False).mean()
C:\Program Files\Anaconda3\lib\site-packages\ipykernel\__main__.py:16: FutureWarning: pd.rolling_mean is deprecated for Series and will be removed in a future version, replace with 
	Series.rolling(window=3,center=False).mean()
C:\Program Files\Anaconda3\lib\site-packages\ipykernel\__main__.py:16: FutureWarning: pd.rolling_mean is deprecated for Series and will be removed in a future version, replace with 
	Series.rolling(window=4,center=False).mean()
C:\Program Files\Anaconda3\lib\site-packages\ipykernel\__main__.py:16: FutureWarning: pd.rolling_mean is deprecated for Series and will be removed in a future version, replace with 
	Series.rolling(window=5,center=False).mean()
C:\Program Files\Anaconda3\lib\site-packages\ipykernel\__main__.py:16: FutureWarning: pd

In [4]:
def mergedataset(datasets):
    useless=['Date','Open','High','Low','Close','Adj Close','Volume']
    for i in range(len(datasets)):
        datasets[i]=datasets[i].drop(useless, axis = 1)
    new_dataset = pd.concat(datasets,axis=1)
    return new_dataset

features=mergedataset(datasets)

features=features[Max:-1]
Target=Target[Max:-1]

Target[Target > 0] = 1
Target[Target <= 0] = 0

X_train=features[:-500]
X_test=features[-500:]
y_train=Target[:-500]
y_test=Target[-500:]

In [11]:
from sklearn.metrics import accuracy_score
def performTimeSeriesCV(X_train, y_train,X_test,y_test, algorithm,params_fix, params):
    """
    """
    
    a=params_fix
    total={}
    for i in range(len(list(params.values())[0])):
        for j in range(len(list(params.values())[1])):
            a[list(params.keys())[0]]=list(params.values())[0][i]
            a[list(params.keys())[1]]=list(params.values())[1][j]
        

            reg=algorithm(**a)
            reg.fit(X_train, y_train)
            y_predict=reg.predict(X_test)
            accuracy=accuracy_score(y_test,y_predict)
            #print(accuracies.mean())
            info=[]
            info.append(list(params.values())[0][i])
            info.append(list(params.values())[1][j])
            total[str(accuracy)]=info
    compare=[]
    for i in total.keys():
        compare.append(float(i))
    Best_accu=max(compare)
    print('Best accuracy:',Best_accu)
    print(list(params.keys())[0],':',total[str(Best_accu)][0])
    print(list(params.keys())[1],':',total[str(Best_accu)][1])
    fin_params=params_fix
    fin_params[list(params.keys())[0]]=total[str(Best_accu)][0]
    fin_params[list(params.keys())[1]]=total[str(Best_accu)][1]
    return fin_params

# Modeling-Xgboost

1 Fix learning rate and number of estimators for tuning tree-based parameters

In [6]:
import pandas as pd
import numpy as np
import xgboost as xgb
from xgboost.sklearn import XGBClassifier
from sklearn import cross_validation, metrics   #Additional scklearn functions

C:\Program Files\Anaconda3\lib\site-packages\sklearn\cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [7]:
def modelfit(alg,X_train, Y_train,useTrainCV=True, cv_folds=5, early_stopping_rounds=50):
    
    if useTrainCV:
        xgb_param = alg.get_xgb_params()
        xgtrain = xgb.DMatrix(X_train, Y_train)
        cvresult = xgb.cv(xgb_param, xgtrain, num_boost_round=alg.get_params()['n_estimators'], nfold=cv_folds,
            metrics='auc', early_stopping_rounds=early_stopping_rounds)
        #alg.set_params(n_estimators=cvresult.shape[0])
    
    #Fit the algorithm on the data
    #alg.fit(X_train, Y_train,eval_metric='rmse')
    print(cvresult.shape[0])

In [8]:
xgb1 = XGBClassifier(
 learning_rate =0.1,
 n_estimators=1000,
 max_depth=5,
 min_child_weight=1,
 gamma=0,
 subsample=0.8,
 colsample_bytree=0.8,
 objective= 'binary:logistic',
 nthread=4,
 scale_pos_weight=1,
 seed=27)

In [9]:
modelfit(xgb1,X_train, y_train)

31


2.2 Tune max_depth and min_child_weight

In [12]:
para=performTimeSeriesCV(X_train, y_train,X_test,y_test,
                         XGBClassifier,{'learning_rate':0.1,'n_estimators':50,'max_depth':5,'min_child_weight':1,'gamma':0,'subsample':0.8,'colsample_bytree':0.8,'objective': 'binary:logistic','nthread':4,'scale_pos_weight':1,'seed':27},
                        {'max_depth':[3,5,7,9,11],'min_child_weight':[1,3,5,7]})

Best accuracy: 0.548
max_depth : 11
min_child_weight : 5


In [13]:
print(para)

{'max_depth': 11, 'gamma': 0, 'nthread': 4, 'learning_rate': 0.1, 'colsample_bytree': 0.8, 'seed': 27, 'min_child_weight': 5, 'n_estimators': 50, 'scale_pos_weight': 1, 'subsample': 0.8, 'objective': 'binary:logistic'}


In [14]:
model=XGBClassifier(**para)
model.fit(X_train, y_train)

XGBClassifier(base_score=0.5, colsample_bylevel=1, colsample_bytree=0.8,
       gamma=0, learning_rate=0.1, max_delta_step=0, max_depth=11,
       min_child_weight=5, missing=None, n_estimators=50, nthread=4,
       objective='binary:logistic', reg_alpha=0, reg_lambda=1,
       scale_pos_weight=1, seed=27, silent=True, subsample=0.8)

In [15]:
y_predict=model.predict(X_test)
print(accuracy_score(y_test,y_predict))

0.548


In [16]:
para=performTimeSeriesCV(X_train, y_train,X_test,y_test,
                         XGBClassifier,{'learning_rate':0.1,'n_estimators':50,'max_depth':5,'min_child_weight':1,'gamma':0,'subsample':0.8,'colsample_bytree':0.8,'objective': 'binary:logistic','nthread':4,'scale_pos_weight':1,'seed':27},
                        {'max_depth':[11,13,15,17,19,21],'min_child_weight':[4,5,6]})

model=XGBClassifier(**para)
model.fit(X_train, y_train)

y_predict=model.predict(X_test)
print(accuracy_score(y_test,y_predict))

Best accuracy: 0.548
max_depth : 11
min_child_weight : 5
0.548


In [17]:
print(para)

{'max_depth': 11, 'gamma': 0, 'nthread': 4, 'learning_rate': 0.1, 'colsample_bytree': 0.8, 'seed': 27, 'min_child_weight': 5, 'n_estimators': 50, 'scale_pos_weight': 1, 'subsample': 0.8, 'objective': 'binary:logistic'}


2.3 Tune gamma

In [25]:
para=performTimeSeriesCV(X_train, y_train,X_test,y_test,
                         XGBClassifier,{'max_depth': 11, 'gamma': 0, 'nthread': 4, 'learning_rate': 0.1, 'colsample_bytree': 0.8, 'seed': 27, 'min_child_weight': 5, 'n_estimators': 50, 'scale_pos_weight': 1, 'subsample': 0.8, 'objective': 'binary:logistic'},
                        {'max_depth':[11], 'gamma':[0.0,0.1,0.2,0.3,0.4,0.5]})

model=XGBClassifier(**para)
model.fit(X_train, y_train)
y_predict=model.predict(X_test)
print(accuracy_score(y_test,y_predict))

Best accuracy: 0.548
max_depth : 11
gamma : 0.0
0.548


In [26]:
print(para)

{'max_depth': 11, 'gamma': 0.0, 'nthread': 4, 'learning_rate': 0.1, 'subsample': 0.8, 'min_child_weight': 5, 'colsample_bytree': 0.8, 'n_estimators': 50, 'seed': 27, 'scale_pos_weight': 1, 'objective': 'binary:logistic'}


re-calibrate the number of boosting rounds for the updated parameters

In [27]:
xgb1 = XGBClassifier(**para)

In [28]:
modelfit(xgb1,X_train, y_train)

50


2.4 Tune subsample and colsample_bytree

In [29]:
para=performTimeSeriesCV(X_train, y_train,X_test,y_test,
                         XGBClassifier,{'max_depth': 11, 'gamma': 0.0, 'nthread': 4, 'learning_rate': 0.1, 'subsample': 0.8, 'min_child_weight': 5, 'colsample_bytree': 0.8, 'n_estimators': 50, 'seed': 27, 'scale_pos_weight': 1, 'objective': 'binary:logistic'},
                        {'subsample':[0.6,0.7,0.8,0.9,1.0], 'colsample_bytree':[0.6,0.7,0.8,0.9,1.0]})

model=XGBClassifier(**para)
model.fit(X_train, y_train)
y_predict=model.predict(X_test)
print(accuracy_score(y_test,y_predict))

Best accuracy: 0.548
colsample_bytree : 0.8
subsample : 0.8
0.548


In [30]:
print(para)

{'max_depth': 11, 'gamma': 0.0, 'nthread': 4, 'learning_rate': 0.1, 'colsample_bytree': 0.8, 'min_child_weight': 5, 'scale_pos_weight': 1, 'n_estimators': 50, 'seed': 27, 'subsample': 0.8, 'objective': 'binary:logistic'}


2.5 Tune Tuning Regularization Parameters

In [36]:
para=performTimeSeriesCV(X_train, y_train,X_test,y_test,
                         XGBClassifier,{'max_depth': 11, 'gamma': 0.0, 'nthread': 4, 'learning_rate': 0.1, 'colsample_bytree': 0.8, 'min_child_weight': 5, 'scale_pos_weight': 1, 'n_estimators': 50, 'seed': 27, 'subsample': 0.8, 'objective': 'binary:logistic'},
                        {'subsample':[0.8], 'reg_alpha':[0.0001,0.001, 0.01, 0.1, 1,10, 100]})

model=XGBClassifier(**para)
model.fit(X_train, y_train)
y_predict=model.predict(X_test)
print(accuracy_score(y_test,y_predict))

Best accuracy: 0.548
reg_alpha : 0.0001
subsample : 0.8
0.548


In [34]:
para=performTimeSeriesCV(X_train, y_train,X_test,y_test,
                         XGBClassifier,{'max_depth': 11, 'gamma': 0.0, 'nthread': 4, 'learning_rate': 0.1, 'colsample_bytree': 0.8, 'min_child_weight': 5, 'scale_pos_weight': 1, 'n_estimators': 50, 'seed': 27, 'subsample': 0.8, 'objective': 'binary:logistic'},
                        {'subsample':[0.8], 'reg_alpha':[0.0001,0.00001, 0.000001, 0.0000001]})

model=XGBClassifier(**para)
model.fit(X_train, y_train)
y_predict=model.predict(X_test)
print(accuracy_score(y_test,y_predict))

Best accuracy: 0.548
reg_alpha : 1e-07
subsample : 0.8
0.548


In [37]:
print(para)

{'max_depth': 11, 'nthread': 4, 'learning_rate': 0.1, 'min_child_weight': 5, 'n_estimators': 50, 'seed': 27, 'scale_pos_weight': 1, 'gamma': 0.0, 'colsample_bytree': 0.8, 'reg_alpha': 0.0001, 'subsample': 0.8, 'objective': 'binary:logistic'}


2.6 Reducing Learning Rate re-calibrate the number of boosting rounds 

In [38]:
para={'max_depth': 11, 'nthread': 4, 'learning_rate': 0.001, 'min_child_weight': 5, 'n_estimators': 50, 'seed': 27, 'scale_pos_weight': 1, 'gamma': 0.0, 'colsample_bytree': 0.8, 'reg_alpha': 0.0001, 'subsample': 0.8, 'objective': 'binary:logistic'}

In [39]:
xgb1 = XGBClassifier(**para)
modelfit(xgb1,X_train, y_train)

50


In [41]:
para=performTimeSeriesCV(X_train, y_train,X_test,y_test,
                         XGBClassifier,{'max_depth': 11, 'nthread': 4, 'learning_rate': 0.1, 'min_child_weight': 5, 'n_estimators': 50, 'seed': 27, 'scale_pos_weight': 1, 'gamma': 0.0, 'colsample_bytree': 0.8, 'reg_alpha': 0.0001, 'subsample': 0.8, 'objective': 'binary:logistic'},
                        {'learning_rate':[0.1,0.01,0.001], 'n_estimators':[50,100,300,500,700,1000,1500,2000]})

model=XGBClassifier(**para)
model.fit(X_train, y_train)
y_predict=model.predict(X_test)
print(accuracy_score(y_test,y_predict))

Best accuracy: 0.548
n_estimators : 50
learning_rate : 0.1
0.548


In [42]:
para=performTimeSeriesCV(X_train, y_train,X_test,y_test,
                         XGBClassifier,{'max_depth': 11, 'nthread': 4, 'learning_rate': 0.1, 'min_child_weight': 5, 'n_estimators': 50, 'seed': 27, 'scale_pos_weight': 1, 'gamma': 0.0, 'colsample_bytree': 0.8, 'reg_alpha': 0.0001, 'subsample': 0.8, 'objective': 'binary:logistic'},
                        {'learning_rate':[0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9], 'n_estimators':[30,50,70,100]})

model=XGBClassifier(**para)
model.fit(X_train, y_train)
y_predict=model.predict(X_test)
print(accuracy_score(y_test,y_predict))

Best accuracy: 0.548
n_estimators : 50
learning_rate : 0.1
0.548
